In [1]:
import re
import math
import sys

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 269
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1313 words
filter frequent words: 540 words
filter letter words: 539 words
filter stop words: 352 words


In [4]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

#### Doc2Vec

In [6]:
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(idx_corpus)]
model = Doc2Vec(tagged_corpus, vector_size=average_doc_size, window=4, min_count=2, epochs=100)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

paragraph_vectors = [model.infer_vector(doc) for doc in idx_corpus]
paragraph_vectors = pandas.DataFrame(paragraph_vectors, dtype=float)

### K-Mean

In [7]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(doc))

    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [8]:
def predict_cluster(bow_corpus, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters).fit(bow_corpus)
    predicted_labels = kmeans.labels_

    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    
    return result

In [13]:
num_clusters = 7
# bow_corpus = get_bow(idx_corpus)
bow_corpus = get_bow(filtered_corpus)
result = predict_cluster(bow_corpus, num_clusters)
bow_corpus.head()

,ทำ,พนักงาน,ข้าว,ดี,น้ำ,อี,เครื่อง,เลิก,กก,งง,...,พลังงาน,คอย,แอร์ไม่,สลิป,คู,ปอง,นวมินทร์,สิบ,ห้า,แจก
0,0.333333,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Result

In [10]:
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4, 5, 6], dtype=int32), array([ 42,  18,  22,  20,   8, 137,  22]))
		percent
0  |	42	1.0


In [11]:
comment_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    print('cluster', change['new'])
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        print(index, value)

comment_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(index=6, options=(1, 2, 3, 4, 5, 6, 7), value=7)

cluster 6
2 ถุงงง  ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้ำยาปรับผ้านุ่มนะ พร้อมขาดดด
3 เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็นส่วนลดได้ด้วย ลองดู เตรียมถุงหนาๆไปเองใช้ได้หลายครั้ง
11 เลือกผัก เลือกแล้ว เลือกอีก พอมาคิดเงินพนักงานยัดผักใส่ถุงจนผักหัก โอ้ยใจ ใจสลาย
13 ราคาสินค้า ตอนโปรโมชั่น บางอันหาไม่เจอ เหมือนพนักงานเอาไปซ่อน ให้ลูกค้าวิ่งไล่หา ไอ้ซั้ซไม่ใช่มอญซ่อนผ้า ปลรปภ บางสาขา หน้าตาดุมาก เวลาเข้าห้างนึกว่าทำไรผิดตลอดเวลา
15 ถุงบางมากนี่เรื่องจริง ลูกค้าบางคนเค้าไม่ได้มีรถส่วนตัว หิ้วขึ้นสองแถว ขึ้นรถเมล์ ก็ต้องเข้าใจเค้าด้วย ขอถุงซ้อนเพิ่มก็กลายเป็นเพิ่มการใช้ถุงพลาสติกเข้าไปอีก ก็ทำให้มันหนากว่านี้หน่อยแล้วใช้ใบเดียวไม่ดีกว่าเหรออย่างโลตัสเอกเพรส ส่วนใหญ่คืออยู่ในเขตหมู่บ้าน ลูกค้าเดินมาซื้อ ขี่จักรยานมาซื้อ ถ้าไม่ขอให้ซ้อนถุงนี่หิ้วกลับไม่ถึงบ้านค่ะ บางทีเลยต้องเข้า 711 ถ้าซื้อของที่มีเหมือนกัน
23 ควย ค่ะ พนักงานแม่งเกือบทุกคนเลย ไม่มีความคิด วิเคราะห์ แยกแยะไม่ได้ว่าเวลาลูกค้าซื้อผัก ซื้อหมูบด น้ำยาล้างจาน น้ำปลา หรืออะไรก็แล้วแต่ ถุงมึงมีใบใหญ่สุดแค่ไหนมึงเอามาใส่แม

In [12]:
token_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

token_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 0})

ToggleButtons(options=(1, 2, 3, 4, 5, 6, 7), value=1)

0 ['พนักงาน', 'พนักงาน', 'โลตัส', 'โลตัส', 'ทำ', 'เอ็กเพลส', 'พนักงาน', 'คน', 'ทำ', 'ขาย', 'คน', 'คน', 'เวลา', 'กิน', 'ข้าว', 'ผม', 'คน', 'พนักงาน', 'เหมือน', 'พนักงาน', 'คน', 'ผม', 'รู้', 'พนักงาน', 'ตั้งใจ', 'ทำ', 'งาน', 'คน']
4 ['ทำ', 'เดือน', 'พนักงาน', 'คน', 'คน', 'คน', 'นึง', 'พัน', 'เช้า', 'เย็น']
7 ['คน', 'คน', 'คน', 'ดี', 'คน', 'ดี', 'ผม', 'โลก', 'ผม', 'ทำ', 'งาน', 'บริการ', 'เหมือน', 'ยืน', 'จุด', 'ยืน', 'รู้', 'พนักงาน', 'เหมือน', 'ปรับปรุง', 'ดี', 'ปรับปรุง', 'ตัว', 'ดี', 'ดี']
8 ['พนักงาน', 'ทำ', 'หน้า', 'ตูด', 'คน', 'เหมือน', 'บังคับ', 'ทำ', 'งาน', 'งั้น']
12 ['มารยาท', 'พนักงาน', 'เซเว่น', 'คุณภาพ', 'ดี', 'นึง', 'เหี้ย', 'คู่']
17 ['พนักงาน', 'เอ็กเพรส', 'สาขา', 'หน้า', 'บูด', 'ทำ', 'งาน', 'กกกกก', 'ถาม', 'รู้', 'เรื่อง', 'ดี']
19 ['สาขา', 'หน้า', 'สาขา', 'แผนก', 'ใด', 'สีหน้า', 'ท่าทาง', 'บริการ', 'เซอร์วิสมายด์', 'ทำ', 'ปล่อย', 'ตอน', 'ทำ', 'ซื้อ', 'ห้าง']
24 ['สาขา', 'แย่', 'ซื้อ', 'ลูก', 'ยืน', 'รอ', 'เงิน', 'ยืน', 'คุย', 'สอง', 'คน', 'หน้า', 'สัก', 'ยืน', 'รอ', 'นาท